# Load synthetic data

In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [52]:
class FeatureScaler(object):
    def __init__(self, train):
        self.mean = train.mean(axis=0)
        self.std = train.std(axis=0)
    
    def transform(self, df):
        return (df - self.mean) / self.std
    
    def inverse_transform(self, df):
        return df * self.std + self.mean
    
    def get_params(self):
        return self.mean, self.std

In [61]:
train = pd.read_csv("./data/heston/train.csv")
val = pd.read_csv("./data/heston/val.csv")
test = pd.read_csv("./data/heston/test.csv")

In [62]:
train.shape, val.shape, test.shape

((900000, 8), (45000, 8), (54411, 8))

In [63]:
train.head()

,Moneyness,Time to Maturity (years),lambda,vbar,eta,rho,v0,iv
0,0.820165,0.219184,5.488135,0.689556,3.947543,-0.586746,0.164776,0.637411
1,0.926490,0.212487,7.151894,0.974517,1.495094,-0.407660,0.337974,0.800476
2,0.834318,0.190719,6.027634,0.808059,4.635460,-0.903236,0.203675,0.693484
3,1.053977,0.226441,5.448832,0.180819,4.405048,-0.373840,0.979871,0.662078
4,0.802749,0.208923,4.236548,0.855412,0.250227,-0.108893,0.200594,0.648292


In [72]:
X_train_origin, Y_train = train.iloc[:, :-1], train.iloc[:, -1]
X_val_origin, Y_val = val.iloc[:, :-1], val.iloc[:, -1]
X_test_origin, Y_test = test.iloc[:, :-1], test.iloc[:, -1]

In [73]:
X_train_origin.head()

,Moneyness,Time to Maturity (years),lambda,vbar,eta,rho,v0
0,0.820165,0.219184,5.488135,0.689556,3.947543,-0.586746,0.164776
1,0.926490,0.212487,7.151894,0.974517,1.495094,-0.407660,0.337974
2,0.834318,0.190719,6.027634,0.808059,4.635460,-0.903236,0.203675
3,1.053977,0.226441,5.448832,0.180819,4.405048,-0.373840,0.979871
4,0.802749,0.208923,4.236548,0.855412,0.250227,-0.108893,0.200594


In [74]:
Y_train.head()

0    0.637411
1    0.800476
2    0.693484
3    0.662078
4    0.648292
Name: iv, dtype: float64

In [75]:
SCALER = FeatureScaler(X_train_origin)
X_train = SCALER.transform(X_train_origin)
X_val = SCALER.transform(X_val_origin)
X_test = SCALER.transform(X_test_origin)

In [76]:
X_train.shape

(900000, 7)

In [77]:
X_train.head()

,Moneyness,Time to Maturity (years),lambda,vbar,eta,rho,v0
0,-0.530693,0.286911,0.167947,0.656690,1.003634,-0.300329,-1.160759
1,0.514788,0.109327,0.744250,1.643183,-0.697421,0.320141,-0.560954
2,-0.391527,-0.467833,0.354822,1.066931,1.480784,-1.396858,-1.026048
3,1.768363,0.479322,0.154333,-1.104483,1.320966,0.437315,1.662005
4,-0.701950,0.014838,-0.265584,1.230858,-1.560880,1.355263,-1.036718


# Neural Network

A neural network returns both the predicted IV value, and corresponding Jacobian matrix

# Deep Calibration

In [80]:
from scipy.stats import truncnorm

def model_parameters_initializer(model='heston', random_seed=None):
    if model == 'heston':
        params = [
            5 * np.random.rand(), # eta
            -1 * np.random.rand(), # rho
            10 * np.random.rand(), # lambda
            np.random.rand(), # vbar
            np.random.rand() #v0
        ]
        names = ['eta', 'rho', 'lambda', 'vbar', 'v0']
    elif model == 'rbergomi' or model == 'rBergomi':
        params = [
            truncnorm(-3, 3, 2.5, 0.5), # eta
            truncnorm(-0.25, 2.25, -0.95, 0.2), # rho
            truncnorm(-1.2, 8.6, 0.07, 0.05), # H
            truncnorm(-2.5, 7, 0.3, 0.1) # v0
        ]
        names = ['eta', 'rho', 'H', 'v0']
    return params, names

In [8]:
def deep_calibration(nn, weights, market_quotes, model='heston', lambd_init=1.0, beta0=0.1, beta1=0.5, max_iter=1000, tol=1e-3):
    # initialize
    params, param_names = model_parameters_initializer(model)
    lambd = lambd_init
    n_params = len(params)
    W = np.diag(weights)
    I = np.eye(n_params)
    n = 0
    # predict
    R = nn.predict(...) - Q # TODO
    J = None # TODO
    J_W = J.T.dot(W)
    delta_mu = np.linalg.pinv(J_W.dot(J) + lambd * I).dot(J_W.dot(R)) # vector size: [m, ]
    while n < max_iter and np.linalg.norm(delta_mu) > tol:
        mu_new = mu + delta_mu
        R_new = nn.predict(...) - Q # TODO
        R_norm = np.linalg.norm(R)
        c_mu = (R_norm - np.linalg.norm(R_new)) / (R_norm - np.linalg.norm(R + J.dot(delta_mu)))
        if c_mu <= beta0:
            # reject delta_mu
            lambd *= 2 # too slow, use greater lambd
        else:
            # accept delta_mu
            mu += delta_mu
            R = R_new
            J = None # TODO
            J_W = J.T.dot(W)
        if c_mu >= beta1:
            lambd /= 2 # too quick, use smaller lambd
        delta_mu = np.linalg.pinv(J_W.dot(J) + lambd * I).dot(J_W.dot(R)) # vector size: [m, ]
        n += 1
    return dict(zip(param_names, mu))